In [2]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator

import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 5
)

query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
query_hello="Hello"

# responses = await parallel_calling_agent._invoke_agent_action_async(query) 
# responses

# agent_action, Observation, done   = await parallel_calling_agent.agent_step_async(query)
parallel_calling_agent.clear_logs()
await parallel_calling_agent.run_agent_episode_async(query=query, reference="The former former president of Korea is older. The sum of their age is 99.")






# parallel_calling_agent.clear_logs()
# parallel_calling_agent.run_agent_trials(num_trials=2, 
# query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
# reference="The former former president of Korea is older. The sum of their age is 99.")




[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=MLB American League MVP last season age)]
[INFO] (step 1-observation) :: [Tool 1-> The ages of former presidents of South Korea vary. Here are the ages of some recent former presidents as of my knowledge cutoff in 2023:

- Moon Jae-in (born January 24, 1953) would be 70 years old.
- Park Geun-hye (born February 2, 1952) would be 71 years old.
- Lee Myung-bak (born December 19, 1941) would be 81 years old.
- Roh Moo-hyun (born August 6, 1946 – May 23, 2009) passed away at the age of 62.
- Kim Dae-jung (born December 3, 1924 – August 18, 2009) passed away at the age of 84.
- Kim Young-sam (born December 20, 1927 – November 22, 2015) passed away at the age of 87.

Please note that these ages are based on the birthdates provided and the year 2023. If you are asking about a specific former president not listed here, please provide their name for a more prec